### IMPORT LIBRARIES & CSV

In [1]:
import pandas as pd
import pymysql

### NORMALIZE DB & SEGMENT TO PROPERLY ENTER INTO TABLES

In [2]:
ello_data = pd.read_csv('nba-elo/nbaallelo.csv')

In [3]:
ello_data['date_game'] = ello_data['date_game'].astype('datetime64[ns]') 

In [4]:
ello_data.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,1946-11-01,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,1946-11-01,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,1946-11-02,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,1946-11-02,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,1946-11-02,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


In [5]:
ello_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [6]:
ello_data.shape

(126314, 23)

### FILTER FOR FORECAST TABLE

In [7]:
forecast_data = ello_data[['team_id', 'date_game', 'game_result', 'elo_i', 'elo_n', 'forecast']]

In [8]:
forecast_data['date_game'] = forecast_data['date_game'].astype(str)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
forecast_data.head()

,team_id,date_game,game_result,elo_i,elo_n,forecast
0,TRH,1946-11-01,L,1300.0000,1293.2767,0.640065
1,NYK,1946-11-01,W,1300.0000,1306.7233,0.359935
2,CHS,1946-11-02,W,1300.0000,1309.6521,0.631101
3,NYK,1946-11-02,L,1306.7233,1297.0712,0.368899
4,DTF,1946-11-02,L,1300.0000,1279.6189,0.640065


### FILTER FOR GAME TABLE

In [10]:
game_data = ello_data[['game_id', 'date_game', 'team_id', 'game_result', 'pts',
                            'opp_id', 'opp_pts']]

In [11]:
game_data.head()

,game_id,date_game,team_id,game_result,pts,opp_id,opp_pts
0,194611010TRH,1946-11-01,TRH,L,66,NYK,68
1,194611010TRH,1946-11-01,NYK,W,68,TRH,66
2,194611020CHS,1946-11-02,CHS,W,63,NYK,47
3,194611020CHS,1946-11-02,NYK,L,47,CHS,63
4,194611020DTF,1946-11-02,DTF,L,33,WSC,50


### FILTER FOR PLAYOFF TABLE

In [12]:
playoff_data = ello_data[ello_data.is_playoffs == 1]

In [13]:
playoff_data['date_game'] = playoff_data['date_game'].astype(str)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
playoff_data.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
662,332,194704020CLR,NBA,0,1947,1947-04-02,61,1,CLR,Rebels,...,41.518227,NYK,Knicks,51,1332.1560,1316.3640,H,W,0.581291,NaN
663,332,194704020CLR,NBA,1,1947,1947-04-02,61,1,NYK,Knicks,...,42.715511,CLR,Rebels,77,1289.1487,1304.9407,A,L,0.418709,NaN
664,333,194704020GSW,NBA,1,1947,1947-04-02,62,1,STB,Bombers,...,47.769810,PHW,Warriors,73,1389.9673,1394.2438,A,L,0.333149,NaN
665,333,194704020GSW,NBA,0,1947,1947-04-02,61,1,PHW,Warriors,...,50.698528,STB,Bombers,68,1369.4114,1365.1349,H,W,0.666851,NaN
666,334,194704020WSC,NBA,0,1947,1947-04-02,61,1,WSC,Capitols,...,59.611252,CHS,Stags,81,1386.6256,1413.9667,H,L,0.791843,NaN


In [15]:
playoff_data = playoff_data[['game_id', 'date_game', 'team_id', 'game_result', 'pts',
                            'opp_id', 'opp_pts']]

In [16]:
playoff_data.head()

,game_id,date_game,team_id,game_result,pts,opp_id,opp_pts
662,194704020CLR,1947-04-02,CLR,W,77,NYK,51
663,194704020CLR,1947-04-02,NYK,L,51,CLR,77
664,194704020GSW,1947-04-02,STB,L,68,PHW,73
665,194704020GSW,1947-04-02,PHW,W,73,STB,68
666,194704020WSC,1947-04-02,WSC,L,65,CHS,81


### FILTER FOR TEAM TABLE

In [17]:
team_data = ello_data.drop_duplicates(subset=['team_id', 'fran_id', 'lg_id', 'year_id'], keep='last')

In [18]:
team_data = team_data[['team_id', 'fran_id', 'lg_id', 'year_id']]

In [19]:
team_data.head()

,team_id,fran_id,lg_id,year_id
640,PIT,Ironmen,NBA,1947
648,DTF,Falcons,NBA,1947
649,TRH,Huskies,NBA,1947
651,PRO,Steamrollers,NBA,1947
657,BOS,Celtics,NBA,1947


In [20]:
team_data.shape

(1487, 4)

In [21]:
# team_data['lg_id'].unique()

In [22]:
# ello_data.dtypes

### CONNECT TO DATABSE

In [23]:
conn = pymysql.connect(host = '34.68.154.83', port = 10596, user = 'aarondm-rw', 
                      passwd = '67de44afc0828ecd6f683dfa488aa797'
#                        , db = 'mysql'
                      , cursorclass = pymysql.cursors.DictCursor)

curr = conn.cursor()

In [24]:
# curr.execute("CREATE DATABASE nba_project")

In [25]:
curr.execute("SHOW DATABASES")

for x in curr:
  print(x)

{'Database': 'employee_training'}
{'Database': 'information_schema'}
{'Database': 'library_circulation'}
{'Database': 'mars'}
{'Database': 'mindful_michigan'}
{'Database': 'mysql'}
{'Database': 'nba_project'}
{'Database': 'newspaper_website'}
{'Database': 'performance_schema'}
{'Database': 'ro_company1'}
{'Database': 'ro_employees'}
{'Database': 'ro_query'}
{'Database': 'ro_recipes'}
{'Database': 'ro_research1'}
{'Database': 'ro_twitter'}
{'Database': 'sakila'}
{'Database': 'sys'}
{'Database': 'world'}


### CREATE TABLE COLUMN NAMES

In [29]:
team = "name, franchise_name, league, year"

In [31]:
forecast = "team_name, game_date, game_result, elo_before_game, elo_after_game, future_prediction"

In [33]:
game = "game_id, date, home_team_name, result_home_team, home_team_pts, away_team_name, away_team_pts"

In [35]:
playoff = "playoff_game_id, date, home_team_name, result_home_team, home_team_pts, away_team_name, away_team_pts"

### CREATE FUNCTION TO CREATE TABLES

In [37]:
def create_tables():
    
    conn = pymysql.connect(host = '34.68.154.83', port = 10596, user = 'aarondm-rw', 
                      passwd = '67de44afc0828ecd6f683dfa488aa797'
#                        , db = 'mysql'
                      , cursorclass = pymysql.cursors.DictCursor)
#     con = authenticate()
    cur = conn.cursor()

    with conn:
        cur.execute("USE nba_project;")
        
        cur.execute(
            
        """CREATE TABLE game (
        id INT AUTO_INCREMENT PRIMARY KEY,
        game_id CHAR(15) NOT NULL,
        date DATE NOT NULL,
        home_team_name VARCHAR(3) NOT NULL,
        result_home_team VARCHAR(1) NOT NULL,
        home_team_pts INT NOT NULL,
        away_team_name VARCHAR(3) NOT NULL,
        away_team_pts INT NOT NULL
        )"""
            
        )

        cur.execute(
            
        """CREATE TABLE playoff (
        id INT AUTO_INCREMENT PRIMARY KEY,
        playoff_game_id INT,
        date DATE NOT NULL,
        home_team_name VARCHAR(3) NOT NULL,
        result_home_team VARCHAR(1) NOT NULL,
        home_team_pts INT NOT NULL,
        away_team_name VARCHAR(3) NOT NULL,
        away_team_pts INT NOT NULL,
        FOREIGN KEY (playoff_game_id) REFERENCES game (id) ON DELETE CASCADE 
        )"""
            
        )

        cur.execute(
        
        """CREATE TABLE team (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(3) NOT NULL,
        franchise_name CHAR(15) NOT NULL, 
        league VARCHAR(3) NOT NULL,
        year INT NOT NULL
        )"""
            
        )
        
        
        cur.execute(
            
        """CREATE TABLE forecast (
        id INT AUTO_INCREMENT PRIMARY KEY,
        team_name VARCHAR(3) NOT NULL,
        game_date DATE NOT NULL,
        game_result VARCHAR(1) NOT NULL,
        elo_before_game FLOAT NOT NULL,
        elo_after_game FLOAT NOT NULL,
        future_prediction FLOAT NOT NULL
        )"""
            
        )
        
    
    cur.close()
    conn.close()

### CREATE FUNCTION TO INSERT DATA INTO COLUMNS
##### DEREK BRUCKNER POINTED OUT A COUPLE RESOURCES TO HELP ME HERE

In [40]:
def insert_data(table_name, cols, database):
    
    conn = pymysql.connect(host = '34.68.154.83', port = 10596, user = 'aarondm-rw', 
                      passwd = '67de44afc0828ecd6f683dfa488aa797'
                      , cursorclass = pymysql.cursors.DictCursor)

    with conn:
        cur = conn.cursor()

        cur.execute("USE nba_project;")
        
        count = 0
        
        for index, row in database.iterrows():
            
            count += 1
            
            if table_name == 'playoff':
                
                print(row)
                                
                sql = """INSERT INTO playoff(playoff_game_id, date, home_team_name, result_home_team, home_team_pts, 
                away_team_name, away_team_pts) VALUES ((SELECT id FROM game WHERE game_id = %s AND home_team_name = %s
                AND id IS NOT NULL), %s, %s, %s, %s, %s, %s)"""
                
                variables = [row[0], row[2], row[1], row[2], row[3], row[4], row[5], row[6]]
                
#                 print(variables)
                
                
                cur.execute(sql, variables)
            
                if count == 99:
                    break 


            else:
            
                sql = 'INSERT INTO ' + str(table_name)+ '(' + cols + ') VALUES' + str(tuple(row))
            
#             print(sql)

                cur.execute(sql)
            
                if count == 1500:
                    break 

        insertion = cur.fetchall()
        
        conn.commit()
        
    cur.close()
    conn.close()

    print(insertion)


In [43]:
create_tables()

In [44]:
insert_data('game', game, game_data)

()


In [45]:
insert_data('team', team, team_data)

()


In [46]:
insert_data('playoff', playoff, playoff_data)

game_id        194704020CLR
date_game        1947-04-02
team_id                 CLR
game_result               W
pts                      77
opp_id                  NYK
opp_pts                  51
Name: 662, dtype: object
['194704020CLR', 'CLR', '1947-04-02', 'CLR', 'W', 77, 'NYK', 51]
game_id        194704020CLR
date_game        1947-04-02
team_id                 NYK
game_result               L
pts                      51
opp_id                  CLR
opp_pts                  77
Name: 663, dtype: object
['194704020CLR', 'NYK', '1947-04-02', 'NYK', 'L', 51, 'CLR', 77]
game_id        194704020GSW
date_game        1947-04-02
team_id                 STB
game_result               L
pts                      68
opp_id                  PHW
opp_pts                  73
Name: 664, dtype: object
['194704020GSW', 'STB', '1947-04-02', 'STB', 'L', 68, 'PHW', 73]
game_id        194704020GSW
date_game        1947-04-02
team_id                 PHW
game_result               W
pts                      73
op

In [47]:
insert_data('forecast', forecast, forecast_data)

()
